In [1]:
import uproot
import numpy as np
import matplotlib.pyplot as plt

from HigherTier import HigherTierFileHelper

In [2]:
#############################
# Let's read the file
#############################

#file = 'stupidUPROOT_0'
#inputFileName = '/Users/isobel/Desktop/DUNE/2024/Hierarchy/HigherTier/files/without2DPFPs/' + file + '.npz'

inputFileName = '/Users/isobel/Desktop/DUNE/2024/Hierarchy/HigherTier/files/stupidUPROOT_0.npz'

In [3]:
#############################
# HACK STUFF
#############################

hackFileName = '/Users/isobel/Desktop/DUNE/2024/Hierarchy/HigherTier/files/handScan/event21.root'
hackTreeFile = uproot.open(hackFileName)
hackTree = hackTreeFile['ccnuselection/ccnusel']
hackBranches = hackTree.arrays()


childConnectionL = np.array(hackBranches['ChildConnectionL'][0])
childConnectionLRatio = np.array(hackBranches['ChildConnectionLRatio'][0])
overshootStartL = np.array(hackBranches['OvershootStartL'][0])
overshootStartDCA = np.array(hackBranches['OvershootStartDCA'][0])
doesChildConnect = np.array(hackBranches['DoesChildConnect'][0])
parentConnectionPointNUpstreamHits = np.array(hackBranches['ParentConnectionPointNUpstreamHits'][0])
parentConnectionPointNDownstreamHits = np.array(hackBranches['ParentConnectionPointNDownstreamHits'][0])
parentConnectionPointNHitRatio = np.array(hackBranches['ParentConnectionPointNHitRatio'][0])
parentConnectionPointEigenValueRatio = np.array(hackBranches['ParentConnectionPointEigenValueRatio'][0])
parentConnectionPointOpeningAngle = np.array(hackBranches['ParentConnectionPointOpeningAngle'][0])
pidLinkType = np.array(hackBranches['PIDLinkType'][0])
pidLinkType_cheat = np.array(hackBranches['PIDLinkType_cheat'][0])

In [4]:
data = np.load(inputFileName)

#############################
# Get file masks
#############################
particle_mask_main = data['particle_mask']
link_mask_main = data['link_mask']

#############################
# Get event-level stuff
#############################
run = data['run']
event = data['event']
subrun = data['subrun']
recoNuVertexX = data['recoNuVertexX']
recoNuVertexY = data['recoNuVertexY']
recoNuVertexZ = data['recoNuVertexZ']

#############################
# True information.. (for cheating)
#############################
pfpTruePDG_main = data['pfpTruePDG']
pfpTrueMomX_main = data['pfpTrueMomX']
pfpTrueMomY_main = data['pfpTrueMomY']
pfpTrueMomZ_main = data['pfpTrueMomZ']
    
#############################
# Get GNN node stuff 
#############################
trackShowerScore_main = data['trackShowerScore']
nHits_main = data['nHits']
charge_main = data['charge']
vertexX_main = data['vertexX']
vertexY_main = data['vertexY']
vertexZ_main = data['vertexZ']
trackEndX_main = data['trackEndX']
trackEndY_main = data['trackEndY']
trackEndZ_main = data['trackEndZ']
showerDirX_main = data['showerDirX']  # not the best direction estimate, placeholder
showerDirY_main = data['showerDirY']
showerDirZ_main = data['showerDirZ']
ivysaurusMuon_main = data['ivysaurusMuon']
ivysaurusProton_main = data['ivysaurusProton']
ivysaurusPion_main = data['ivysaurusPion']
ivysaurusElectron_main = data['ivysaurusElectron']
ivysaurusPhoton_main = data['ivysaurusPhoton']
trackLength_main = data['trackLength']
displacement_main = data['displacement']
dca_main = data['dca']
nuVertexEnergyAsymmetry_main = data['nuVertexEnergyAsymmetry']
nuVertexEnergyWeightedMeanRadialDistance_main = data['nuVertexEnergyWeightedMeanRadialDistance']

#############################
# Get higher tier link stuff
#############################
parentPFPIndices_main = data['parentPFPIndices']
childPFPIndices_main = data['childPFPIndices']
parentTrackScore_main = data['parentTrackScore']
parentNuVertexSeparation_main = data['parentNuVertexSeparation']
childNuVertexSeparation_main = data['childNuVertexSeparation']
parentEndRegionNHits_main = data['parentEndRegionNHits']
parentEndRegionNParticles_main = data['parentEndRegionNParticles']
parentEndRegionRToWall_main = data['parentEndRegionRToWall']
vertexSeparation_main = data['vertexSeparation']
separation3D_main = data['separation3D']
chargeRatio_main = data['chargeRatio']
pidLinkType_main = data['pidLinkType']
openingAngle_main = data['openingAngle']
trackShowerLinkType_main = data['trackShowerLinkType']

#############################
# Get node truth
#############################
trueTrackID_main = data['trueTrackID']
trueVisibleParentTrackID_main = data['trueVisibleParentTrackID']
trueVisibleGeneration_main = data['trueVisibleGeneration']

#############################
# Get link truth
#############################
trueParentChildLink_main = data['trueParentChildLink']

#############################
# How many entries are we working with?
#############################
nEntries = particle_mask_main.shape[0]
print('We are working with:', nEntries, 'entries')
print(link_mask_main.shape[0])

#############################
# Network scores
#############################
trueTrackID_out = []
trueVisibleGeneration_out = []
trueVisibleParentTrackID_out = []

particleIndex_out = []
primaryScores_out = []

higherTierChildIndex_out = []
higherTierParentIndex_out = []
higherTierParentTrackID_out = []
higherTierScores_out = []

#############################
# GNN config
#############################

modeDict = {
    "ADD_NEUTRINO"                 : True,
    "CHEAT_DIRECTION"              : True,
    "CHEAT_PID"                    : True, 
    "MAKE_PARTICLE_PARTICLE_LINKS" : True,
    "EDGE_FRACTION"                : 0.8,
    "DO_NORMALISATION"             : True,
    "IS_PRIMARY_TRAINING"          : False,
    "IS_HIGHER_TIER_TRAINING"      : False,
    "MAX_NODE_CLASS"               : 4
}

We are working with: 96300 entries
96300


In [29]:
#iEvent = 



for iEvent in range(110, particle_mask_main.shape[0]) :
    
    trackIDs = trueTrackID_main[iEvent][particle_mask_main[iEvent]]
    trueVisibleParentTrackIDs = trueVisibleParentTrackID_main[iEvent][particle_mask_main[iEvent]]
    
    # Do we have a split particle?
    for trackID in trackIDs :

        instance = np.count_nonzero(trackIDs == trackID)

        if (instance > 1) : 
            for iParticle in range(0, trackIDs.shape[0]) :
        
                # Are we looking at a segment of the split particle?
                if (trackIDs[iParticle] == trueVisibleParentTrackIDs[iParticle]) :
                    continue
                    
                if (trueVisibleParentTrackIDs[iParticle] == trackID) :
                    print(iEvent)
                    print(iParticle)
                    raise Exception("Date provided can't be in the past")
        
    



159
2


Exception: Date provided can't be in the past

In [30]:
iEvent = 159

print('run:', run[iEvent])
print('event:', event[iEvent])
print('trueTrackID_main:', trueTrackID_main[iEvent][particle_mask_main[iEvent]])
print('trackShowerScore_main:', trackShowerScore_main[iEvent][particle_mask_main[iEvent]])
print('trueVisibleParentTrackID_main:', trueVisibleParentTrackID_main[iEvent][particle_mask_main[iEvent]])

run: 1427
event: 96260
trueTrackID_main: [ 5  5 79]
trackShowerScore_main: [0.66435975 0.70246291 0.5539934 ]
trueVisibleParentTrackID_main: [0 5 5]


In [ ]:
###############################################
# Find Primaries - Let's create our graphs!
###############################################

for iEvent in [242] :
    
    ########################################
    # Read our input arrays
    ########################################
    particle_mask_event = particle_mask_main[iEvent]
    link_mask_event = link_mask_main[iEvent]

    # For higher tier network
    higherTierDict = {
        "parentTrackScore"           : parentTrackScore_main[iEvent][link_mask_event], \
        "parentNuVertexSeparation"   : parentNuVertexSeparation_main[iEvent][link_mask_event], \
        "childNuVertexSeparation"    : childNuVertexSeparation_main[iEvent][link_mask_event], \
        "parentEndRegionNHits"       : parentEndRegionNHits_main[iEvent][link_mask_event], \
        "parentEndRegionNParticles"  : parentEndRegionNParticles_main[iEvent][link_mask_event], \
        "parentEndRegionRToWall"     : parentEndRegionRToWall_main[iEvent][link_mask_event], \
        "vertexSeparation"           : vertexSeparation_main[iEvent][link_mask_event], \
        "separation3D"               : separation3D_main[iEvent][link_mask_event], \
        "chargeRatio"                : chargeRatio_main[iEvent][link_mask_event], \
        "pidLinkType"                : pidLinkType_main[iEvent][link_mask_event], \
        "openingAngle"               : openingAngle_main[iEvent][link_mask_event], \
        "trackShowerLinkType"        : trackShowerLinkType_main[iEvent][link_mask_event], \
        "trueParentChildLink"        : trueParentChildLink_main[iEvent][link_mask_event], \
        "parentPFPIndices"           : parentPFPIndices_main[iEvent][link_mask_event], \
        "childPFPIndices"            : childPFPIndices_main[iEvent][link_mask_event]
    }
        
    # For GNN
    eventDict = {
        "recoNuVertexX"                            : recoNuVertexX[iEvent], \
        "recoNuVertexY"                            : recoNuVertexY[iEvent], \
        "recoNuVertexZ"                            : recoNuVertexZ[iEvent], \
        "nParticles"                               : np.count_nonzero(particle_mask_event), \
        "trackShowerScore"                         : trackShowerScore_main[iEvent][particle_mask_event], \
        "nHits"                                    : nHits_main[iEvent][particle_mask_event], \
        "charge"                                   : charge_main[iEvent][particle_mask_event], \
        "vertexX"                                  : vertexX_main[iEvent][particle_mask_event], \
        "vertexY"                                  : vertexY_main[iEvent][particle_mask_event], \
        "vertexZ"                                  : vertexZ_main[iEvent][particle_mask_event], \
        "trackEndX"                                : trackEndX_main[iEvent][particle_mask_event], \
        "trackEndY"                                : trackEndY_main[iEvent][particle_mask_event], \
        "trackEndZ"                                : trackEndZ_main[iEvent][particle_mask_event], \
        "showerDirX"                               : showerDirX_main[iEvent][particle_mask_event], \
        "showerDirY"                               : showerDirY_main[iEvent][particle_mask_event], \
        "showerDirZ"                               : showerDirZ_main[iEvent][particle_mask_event], \
        "ivysaurusMuon"                            : ivysaurusMuon_main[iEvent][particle_mask_event], \
        "ivysaurusProton"                          : ivysaurusProton_main[iEvent][particle_mask_event], \
        "ivysaurusPion"                            : ivysaurusPion_main[iEvent][particle_mask_event], \
        "ivysaurusElectron"                        : ivysaurusElectron_main[iEvent][particle_mask_event], \
        "ivysaurusPhoton"                          : ivysaurusPhoton_main[iEvent][particle_mask_event], \
        "trackLength"                              : trackLength_main[iEvent][particle_mask_event], \
        "displacement"                             : displacement_main[iEvent][particle_mask_event], \
        "dca"                                      : dca_main[iEvent][particle_mask_event], \
        "isNeutrinoPDG"                            : np.zeros(pfpTruePDG_main[iEvent][particle_mask_event].shape), \
        "nuVertexEnergyAsymmetry"                  : nuVertexEnergyAsymmetry_main[iEvent][particle_mask_event], \
        "nuVertexEnergyWeightedMeanRadialDistance" : nuVertexEnergyWeightedMeanRadialDistance_main[iEvent][particle_mask_event], \
        "trueTrackID"                              : trueTrackID_main[iEvent][particle_mask_event], \
        "trueVisibleParentTrackID"                 : trueVisibleParentTrackID_main[iEvent][particle_mask_event], \
        "trueMomX"                                 : pfpTrueMomX_main[iEvent][particle_mask_event], \
        "trueMomY"                                 : pfpTrueMomY_main[iEvent][particle_mask_event], \
        "trueMomZ"                                 : pfpTrueMomZ_main[iEvent][particle_mask_event], \
        "truePDG"                                  : pfpTruePDG_main[iEvent][particle_mask_event], \
        "trueVisibleGeneration"                    : trueVisibleGeneration_main[iEvent][particle_mask_event]
    }
    
    ########################################
    # Get input for higher tiers network
    ########################################
    #variables, y, parentPFPIndices, childPFPIndices = HigherTierFileHelper.readEvent(higherTierDict)
        
    #linkIndex = HigherTierFileHelper.getLinkIndex(parentPFPIndices, childPFPIndices, 0, 1) 
        
        
    print('run:', run[iEvent])
    print('subrun:', subrun[iEvent])
    print('event:', event[iEvent])
    print('')
        
        
#     print("trueTrackID:", eventDict["trueTrackID"])
#     print("truePDG:", eventDict["truePDG"])
#     print("trueVisibleGeneration:", eventDict["trueVisibleGeneration"])
#     print("vertexX:", eventDict["vertexX"])
#     print("trackShowerScore:", eventDict["trackShowerScore"])



    


In [ ]:
print('RecoPFPTrueTrackID:', hackBranches["RecoPFPTrueTrackID"])
print('RecoPFPTruePDG:', hackBranches["RecoPFPTruePDG"])

In [ ]:
PARENT_INDEX = 0
CHILD_INDEX = 2

LINK_INDEX = 23#HigherTierFileHelper.getLinkIndex(np.array(hackBranches["ParentPFPIndex"][0]), np.array(hackBranches["ChildPFPIndex"][0]), PARENT_INDEX, CHILD_INDEX)


print('')
print('')
print('')
print('')
    
print('childPFPIndices:', hackBranches["ChildPFPIndex"][0][LINK_INDEX])   
print('parentPFPIndices', hackBranches["ParentPFPIndex"][0][LINK_INDEX])
print('Link index:', LINK_INDEX)

print('TrueParentChildLink:', hackBranches["TrueParentChildLink"][0][LINK_INDEX])  
print('IsHigherTierTrainingLink:', hackBranches["IsHigherTierTrainingLink"][0][LINK_INDEX])  

print('')
print('parentTrackShowerScore', hackBranches["ParentTrackScore"][0][LINK_INDEX])
print('parentNuVertexSeparation', hackBranches["ParentNuVertexSeparation"][0][LINK_INDEX])
print('childNuVertexSeparation', hackBranches["ChildNuVertexSeparation"][0][LINK_INDEX])
print('parentEndRegionNHits', hackBranches["ParentEndRegionNHits"][0][LINK_INDEX])
print('parentEndRegionNParticles', hackBranches["ParentEndRegionNParticles"][0][LINK_INDEX])
print('parentEndRegionRToWall', hackBranches["ParentEndRegionRToWall"][0][LINK_INDEX])
print('vertexSeparation', hackBranches["VertexSeparation"][0][LINK_INDEX])
print('separation3D:', hackBranches["Separation3D"][0][LINK_INDEX])
print('chargeRatio', hackBranches["ChargeRatio"][0][LINK_INDEX])
print('openingAngle', hackBranches["OpeningAngle"][0][LINK_INDEX])
print('trackShowerLinkType', hackBranches["TrackShowerLinkType"][0][LINK_INDEX])






print('pidLinkType:', pidLinkType[LINK_INDEX])
print('pidLinkType_cheat:', pidLinkType_cheat[LINK_INDEX])
print('childConnectionL:', childConnectionL[LINK_INDEX])

print('ChildConnectionLRatio:', childConnectionLRatio[LINK_INDEX])
print('UnderOvershootL:', underOvershootL[LINK_INDEX])
print('UnderOvershootDCA:', underOvershootDCA[LINK_INDEX])

print('DoesChildConnect:', doesChildConnect[LINK_INDEX])
print('parentConnectionPointNUpstreamHits:', parentConnectionPointNUpstreamHits[LINK_INDEX])
print('parentConnectionPointNDownstreamHits:', parentConnectionPointNDownstreamHits[LINK_INDEX])
print('parentConnectionPointNHitRatio:', parentConnectionPointNHitRatio[LINK_INDEX])
print('parentConnectionPointEigenValueRatio:', parentConnectionPointEigenValueRatio[LINK_INDEX])
print('parentConnectionPointOpeningAngle:', parentConnectionPointOpeningAngle[LINK_INDEX])

In [6]:
for i in range(childConnectionLRatio.shape[0]) :
    print(i, 'Is true parent?', hackBranches['TrueParentChildLink'][0][i])
    print(i, 'Is true training link?', hackBranches['IsHigherTierTrainingLink'][0][i])
    print(i, ':', '{:.2f}'.format(childConnectionLRatio[i]))
    print(i, 'Parent index:', hackBranches["ParentPFPIndex"][0][i])
    print(i, 'Child index:', hackBranches["ChildPFPIndex"][0][i])
    print('--------------------')
    


0 Is true parent? False
0 Is true training link? True
0 : -9999.00
0 Parent index: 2
0 Child index: 0
--------------------
1 Is true parent? False
1 Is true training link? True
1 : -9999.00
1 Parent index: 2
1 Child index: 1
--------------------
2 Is true parent? False
2 Is true training link? True
2 : -9999.00
2 Parent index: 2
2 Child index: 3
--------------------
3 Is true parent? False
3 Is true training link? True
3 : -9999.00
3 Parent index: 2
3 Child index: 4
--------------------
4 Is true parent? False
4 Is true training link? True
4 : -9999.00
4 Parent index: 2
4 Child index: 5
--------------------
5 Is true parent? False
5 Is true training link? True
5 : -9999.00
5 Parent index: 2
5 Child index: 6
--------------------
6 Is true parent? False
6 Is true training link? True
6 : -9999.00
6 Parent index: 3
6 Child index: 0
--------------------
7 Is true parent? False
7 Is true training link? True
7 : -9999.00
7 Parent index: 3
7 Child index: 1
--------------------
8 Is true parent

In [ ]:
#print('lUnderOvershoot:', lUnderOvershoot[0])


parent = np.array(hackBranches["ParentPFPIndex"][0])[doesChildConnect[0] == True]
child = np.array(hackBranches["ChildPFPIndex"][0])[doesChildConnect[0] == True]
true = np.array(hackBranches["TrueParentChildLink"][0])[doesChildConnect[0] == True]
dca = np.array(hackBranches["ChildDCA"][0])[doesChildConnect[0] == True]
lratio = np.array(hackBranches["ChildConnectionLRatio"][0])[doesChildConnect[0] == True]

for i in range(len(parent)) :
    print('--------------------')
    print('parent:', parent[i])
    print('child:', child[i])
    print('true:', true[i])
    print('dca:', dca[i])
    print('lratio:', lratio[i])
    print('--------------------')

print(np.count_nonzero(true))

to_plot = np.array(doesChildConnect[0])
to_plot = to_plot[to_plot > 0]

plt.hist(to_plot)